In [10]:
using Revise
using GeometryBasics: Vec3f, Point3f
using LinearAlgebra
using GLMakie
using StaticArrays
using VMRobotControl
using VMRobotControl.Splines: CubicSpline
using DifferentialEquations
using MeshIO
include("../../functions.jl")

generate_q_init (generic function with 1 method)

## Importing ShadowHand URDF

### URDF Parsing

In [19]:
using FileIO, UUIDs
try
    FileIO.add_format(format"DAE", (), ".dae", [:DigitalAssetExchangeFormatIO => UUID("43182933-f65b-495a-9e05-4d939cea427d")])
catch
end

cfg = URDFParserConfig(;suppress_warnings=true) # This is just to hide warnings about unsupported URDF features
module_path = joinpath(splitpath(splitdir(pathof(VMRobotControl))[1])[1:end-1])
robot = parseURDF(joinpath(module_path, "URDFs/sr_description/sr_hand_vm_compatible.urdf"), cfg)

24DOF Mechanism{Float64} "shadowhand_motor" with 33 frames, 32 joints, 55 coordinates, 82 components

### Kinematic computations : Obtaining the three points

In [3]:
m = compile(robot)
kcache = new_kinematics_cache(m)  

rh_fftip_frame_id = get_compiled_frameID(m, "rh_fftip")
fftip_transform = get_transform(kcache, rh_fftip_frame_id)
display(fftip_transform.origin)

3-element SVector{3, Float64} with indices SOneTo(3):
  0.033
 -0.01
  0.43801000000000007

### Hand Visualization

In [ ]:
# Target transform 
R = [1.0 0.0 0.0; 
     0.0 1.0 0.0;
     0.0 0.0 1.0]

t = [0.0375, -0.08, 0.39]  # Example translation vector

# Construct the homogeneous transformation matrix
T1 = [R t; 0 0 0 1]

t2 = [0.009, -0.093, 0.362]

T2 = [R t2; 0 0 0 1]

4×4 Matrix{Float64}:
 1.0  0.0  0.0   0.009
 0.0  1.0  0.0  -0.093
 0.0  0.0  1.0   0.362
 0.0  0.0  0.0   1.0

In [51]:
# Compile robot model
m = compile(robot)
kcache = Observable(new_kinematics_cache(m))  # This stores the robot's joint state

# Create the figure
fig = Figure(size=(800, 600))

# Create interactive 3D scene
ls = LScene(fig[1, 1]; show_axis=true)  # 3D interactive scene

# Attach interactive camera controls
cam3d!(ls)  # Enables mouse interaction (rotate, zoom, pan)

# Plot the robot's visuals
robotvisualize!(ls, kcache)
#display_frame(m, ls, "rh_ffmiddle")
#display_frame(m, ls, "world")
display_transform(ls, T1)
display_transform(ls, T2)

# Display figure
display(fig)

GLMakie.Screen(...)

In [13]:
show(IOContext(stdout, :limit => false),  MIME("text/plain"), frames(robot))

33-element Vector{String}:
 "world"
 "rh_forearm"
 "rh_wrist"
 "rh_palm"
 "rh_manipulator"
 "rh_imu"
 "rh_ffknuckle"
 "rh_ffproximal"
 "rh_ffmiddle"
 "rh_ffdistal"
 "rh_fftip"
 "rh_mfknuckle"
 "rh_mfproximal"
 "rh_mfmiddle"
 "rh_mfdistal"
 "rh_mftip"
 "rh_rfknuckle"
 "rh_rfproximal"
 "rh_rfmiddle"
 "rh_rfdistal"
 "rh_rftip"
 "rh_lfmetacarpal"
 "rh_lfknuckle"
 "rh_lfproximal"
 "rh_lfmiddle"
 "rh_lfdistal"
 "rh_lftip"
 "rh_thbase"
 "rh_thproximal"
 "rh_thhub"
 "rh_thmiddle"
 "rh_thdistal"
 "rh_thtip"

## Creating the Virtual Mechanism System

### Gravity Compensation, Joint Limits and Joint Damping

In [20]:
# GRAVITY COMPENSATION

add_gravity_compensation!(robot, VMRobotControl.DEFAULT_GRAVITY)

# JOINT DAMPING and LIMIT SPRINGS

joint_limits = cfg.joint_limits

for joint_id in keys(joints(robot))
    limits = joint_limits[joint_id]
    isnothing(limits) && continue
    add_coordinate!(robot, JointSubspace(joint_id);  id="$(joint_id)_coord")
    @assert ~isnothing(limits.lower) && ~isnothing(limits.upper)
    add_deadzone_springs!(robot, 50.0, (limits.lower+0.1, limits.upper-0.1), "$(joint_id)_coord")
    add_component!(robot, LinearDamper(0.01, "$(joint_id)_coord"); id="$(joint_id)_damper")
end

### Addition of the multiples springs/dampers

In [41]:
vms = VirtualMechanismSystem("myShadowVMS", robot)
root = root_frame(vms.robot)

K = SMatrix{3, 3}(100., 0., 0., 0., 100., 0., 0., 0., 100.)
D = SMatrix{3, 3}(10., 0., 0., 0., 10.0, 0., 0., 0., 10.)

ff_target = SVector(0.0375, -0.08, 0.37)
mf_target = SVector(0.009, -0.093, 0.342)
th_target = (ff_target + mf_target)/2

#Lightly constraint some joints to avoid unwanted motions
add_component!(vms, LinearSpring(1.0, ".robot.rh_LFJ5_coord"); id = "lf j5 angular spring")

add_coordinate!(vms, ConstCoord(ff_target); id="ff target position")
add_coordinate!(vms, FrameOrigin(".robot.rh_fftip"); id="ff position")
add_coordinate!(vms, CoordDifference("ff position", "ff target position"); id="ff target dist")
add_coordinate!(vms, CoordNorm("ff target dist"); id="ff target norm")
add_coordinate!(vms, ConstCoord(0.1); id="ff spring length")
add_coordinate!(vms, CoordDifference("ff target norm", "ff spring length"); id="ff position error")

add_component!(vms, LinearSpring(100.0, "ff position error"); id="ff positioning spring")
add_component!(vms, LinearDamper(10.0, "ff position error"); id="ff positioning damper")

add_coordinate!(vms, ConstCoord(mf_target); id="mf target position")
add_coordinate!(vms, FrameOrigin(".robot.rh_mftip"); id="mf position")
add_coordinate!(vms, CoordDifference("mf position", "mf target position"); id="mf target dist")
add_coordinate!(vms, CoordNorm("mf target dist"); id="mf target norm")
add_coordinate!(vms, ConstCoord(0.1); id="mf spring length")
add_coordinate!(vms, CoordDifference("mf target norm", "mf spring length"); id="mf position error")

add_component!(vms, LinearSpring(100.0, "mf position error"); id="mf positioning spring")
add_component!(vms, LinearDamper(10.0, "mf position error"); id="mf positioning damper")

add_coordinate!(vms, ConstCoord(th_target); id="th target position")
add_coordinate!(vms, FrameOrigin(".robot.rh_thtip"); id="th position")
add_coordinate!(vms, CoordDifference("th position", "th target position"); id="th target dist")
add_coordinate!(vms, CoordNorm("th target dist"); id="th target norm")
add_coordinate!(vms, ConstCoord(0.07); id="th spring length")
add_coordinate!(vms, CoordDifference("th target norm", "th spring length"); id="th position error")

add_component!(vms, LinearSpring(100.0, "th position error"); id="th positioning spring")
add_component!(vms, LinearDamper(10.0, "th position error"); id="th positioning damper")


"th positioning damper"

## Simulating the Robot

### Setting Up the Simulation

In [42]:
using Logging: global_logger
using TerminalLoggers: TerminalLogger
global_logger(TerminalLogger())

"""
Update the vms based on the "coord", which is a coordinate varying between 
0 and 1, 0 being the starting position of the medium wrap motion while 1 being 
the end
"""
function update_prismatic_2_coord(args, cache, coord)
    ff_spring_length_id, mf_spring_length_id, th_spring_length_id = args

    #update the length of the spring between the target and the ff finger
    length_max = 0.1
    length_min = 0.0
    length_value = length_max - (length_max - length_min)*coord 
    cache[ff_spring_length_id] = remake(cache[ff_spring_length_id] ; coord_data = ConstCoord(length_value))

    #update the length of the spring between the target and the mf finger
    length_max = 0.1
    length_min = 0.0
    length_value = length_max - (length_max - length_min)*coord 
    cache[mf_spring_length_id] = remake(cache[mf_spring_length_id] ; coord_data = ConstCoord(length_value))

    #update the length of the spring between the target and the thumb
    length_max = 0.07
    length_min = 0.0
    length_value = length_max - (length_max - length_min)*coord 
    cache[th_spring_length_id] = remake(cache[th_spring_length_id] ; coord_data = ConstCoord(length_value))

    nothing
end

function f_setup(cache) 
    return (get_compiled_coordID(cache, "ff spring length") , get_compiled_coordID(cache, "mf spring length"), get_compiled_coordID(cache, "th spring length"))
end

function f_control(cache, t, args, extra)

    t_end = 5. #we want to reach the end position at t = t_end 
    
    if t <= t_end 
        coord_value = t/t_end
        update_prismatic_2_coord(args, cache, coord_value)
    end
end

tspan = (0., 6.)
vms_compiled = compile(vms)

q_init = generate_q_init(vms_compiled; rf=true, lf=true)
q = (q_init, zero_q(vms_compiled.virtual_mechanism)) # Robot joint angle, vm joint angles
q̇ = (zero_q̇(vms_compiled.robot), zero_q̇(vms_compiled.virtual_mechanism)) # Robot joint velocity, vm joint velocities

g = VMRobotControl.DEFAULT_GRAVITY
dcache = new_dynamics_cache(vms_compiled)
prob = get_ode_problem(dcache, g, q, q̇, tspan; f_setup, f_control)
@info "Simulating shadow robot with prismatic 2 grasp"

sol = solve(prob, Tsit5(), progress=true; maxiters=1e6, abstol=1e-5, reltol=1e-5);

[ Info: Simulating shadow robot with prismatic 2 grasp
ODE   0%|█                                              |  ETA: N/A
ODE   2%|█                                              |  ETA: 0:01:20
ODE   3%|██                                             |  ETA: 0:01:30
ODE   5%|███                                            |  ETA: 0:01:38
ODE   7%|████                                           |  ETA: 0:02:01
ODE   8%|████                                           |  ETA: 0:02:05
ODE  10%|█████                                          |  ETA: 0:01:56
ODE  12%|██████                                         |  ETA: 0:01:48
ODE  13%|███████                                        |  ETA: 0:01:43
ODE  15%|███████                                        |  ETA: 0:01:38
ODE  16%|████████                                       |  ETA: 0:01:34
ODE  18%|█████████                                      |  ETA: 0:01:32
ODE  20%|██████████                                     |  ETA: 0:01:31
ODE  21%|████

### Visualizing the Results

In [52]:
fig = Figure(; size=(720, 720), figure_padding=0)
display(fig)
ls = LScene(fig[1, 1]; show_axis=false)
cam = cam3d!(ls; center=true)
cam.lookat[] = [0.025, 0., 0.24]
cam.eyeposition[] = [-0.25, -0.6, 0.62]

plotting_t = Observable(0.0)
plotting_kcache = Observable(new_kinematics_cache(compile(vms)))
robotvisualize!(ls, plotting_kcache)

#cartID = get_compiled_coordID(plotting_kcache[], ".virtual_mechanism.cart position")
#scatter!(ls, plotting_kcache, cartID; color=:red, marker=:rect, markersize=5)
scatter!(ls, ff_target ; color=:red, marker=:rect, markersize=5)
scatter!(ls, mf_target ; color=:red, marker=:rect, markersize=5)
scatter!(ls, th_target ; color=:red, marker=:rect, markersize=5)

# Draw the line on the scene
line_point_1 = [-0.0195, -0.106, 0.314]
line_point_2 = [0.066, -0.067, 0.398]
lines!(ls, [line_point_1[1], line_point_2[1]], [line_point_1[2], line_point_2[2]], [line_point_1[3], line_point_2[3]], color=:black, linewidth=3)


savepath = joinpath(module_path, "docs/src/assets/shadowhand_prismatic_2.mp4")
display(fig)
animate_robot_odesolution(fig, sol, plotting_kcache, savepath; t=plotting_t);